In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

tf.random.set_seed(42)

2023-04-02 17:22:39.977710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 17:22:41.041032: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-02 17:22:41.041077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-02 17:22:41.041081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
data = pd.read_csv('data/file.csv', index_col=0)
data.head()


,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [3]:
dataset = tf.data.Dataset.from_tensor_slices((data.tweets, data.labels))

2023-04-02 17:22:42.303987: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-04-02 17:22:42.304011: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
for X_batch, y_batch in dataset.batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print(review)
        print(label)

b'ChatGPT: Optimizing Language Models for Dialogue https://t.co/K9rKRygYyn @OpenAI'
b'neutral'
b'Try talking with ChatGPT, our new AI system which is optimized for dialogue. Your feedback will help us improve it. https://t.co/sHDm57g3Kr'
b'good'


In [5]:
def preprocess(X_batch, y_batch):
    #ogranicznie znakow do 300
    X_batch = tf.strings.substr(X_batch, 0, 300)
    #usuniecie linkow
    X_batch = tf.strings.regex_replace(X_batch, r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", b" ") 
    #usuniecie \n
    X_batch = tf.strings.regex_replace(X_batch, r"\\n", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)

    y_batch = tf.where(y_batch=="good", 2   , tf.where(y_batch=="neutral", 1, 0));

    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [6]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in dataset.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))


In [7]:
vocabulary.most_common()[:25]

[(b'<pad>', 6225388),
 (b'ChatGPT', 191857),
 (b'to', 128447),
 (b'the', 125763),
 (b'a', 101035),
 (b'and', 85379),
 (b'is', 80733),
 (b'of', 74310),
 (b'I', 68907),
 (b'it', 62526),
 (b'AI', 48598),
 (b'for', 47855),
 (b'in', 46016),
 (b'with', 38492),
 (b'that', 32069),
 (b'on', 31438),
 (b'you', 29917),
 (b'this', 25562),
 (b'about', 24506),
 (b'be', 23174),
 (b'can', 21524),
 (b'by', 19839),
 (b'OpenAI', 19811),
 (b'my', 19711),
 (b'The', 19253)]

In [8]:


vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]



In [9]:


word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"OpenAI my about be can".split():
    print(word_to_id.get(word) or vocab_size)



22
23
18
19
20


In [10]:


words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)



In [11]:


def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = dataset.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)



In [12]:


for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)



tf.Tensor(
[[   1 1684  696 ...    0    0    0]
 [ 763  278   13 ...    0    0    0]
 [   1 1684  696 ...    0    0    0]
 ...
 [ 142   12    1 ...    0    0    0]
 [   1    6   47 ...    0    0    0]
 [ 763  278   13 ...    0    0    0]], shape=(32, 45), dtype=int64)
tf.Tensor([1 2 1 2 0 2 0 2 2 1 0 1 1 1 1 1 1 0 1 1 1 2 1 1 2 1 2 1 1 1 2 2], shape=(32,), dtype=int32)


In [13]:


embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(3, activation="sigmoid")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)



Epoch 1/5
6853/6853 [==============================] - 560s 81ms/step - loss: 0.3848 - accuracy: 0.8575
Epoch 2/5
 461/6853 [=>............................] - ETA: 7:49 - loss: 0.2892 - accuracy: 0.9033

KeyboardInterrupt: 